In [ ]:
%matplotlib inline
import pathlib
import warnings
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from torch import nn
from torch.utils import data

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
seq = 32
hidden_size = 32
num_layers = 8
batch_size = 32
epochs = 100

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("*********************")
print(torch.cuda.get_device_name(0))
print(device)

In [ ]:
def load_data():
    train = pd.read_excel('./dataset/train.xlsx')
    test = pd.read_excel('./dataset/test.xlsx')

    time_columns = ['time']
    X_columns = [col for col in train.columns if col not in ['time','CODeff']]
    Y_columns =['CODeff']

    x_train = train[X_columns]
    y_train = train[Y_columns]
    x_test = test[X_columns]
    y_test = test[Y_columns]
    return x_train, y_train, x_test, y_test, X_columns, Y_columns


def split_data(train_sca, test_sca, seq, X_columns, Y_columns):  # 47预测1
    data_raw = pd.concat([train_sca, test_sca])

    data = []
    for index in range(len(data_raw) - seq):
        data.append(data_raw[index:index + seq])
    data = np.array(data)
    # print(len(data))  #14640
    test_set_size = len(test_sca)  # 96
    train_set_size = data.shape[0] - test_set_size

    # 第一个-1是47预测1
    # 第二个1是不计time，-1是只预测AQ
    x_train = data[:train_set_size, :-1, :len(X_columns)].astype('float32')
    y_train = data[:train_set_size, -1, -len(Y_columns)].reshape(-1, len(Y_columns)).astype('float32')

    x_test = data[train_set_size:, :-1, :len(X_columns)].astype('float32')
    y_test = data[train_set_size:, -1, -len(Y_columns)].reshape(-1, len(Y_columns)).astype('float32')

    return torch.tensor(x_train), torch.tensor(y_train), torch.tensor(x_test), torch.tensor(y_test)


def preprocess(x_train, y_train, x_test, y_test, X_columns, Y_columns):
    scaler1 = MinMaxScaler(feature_range=(0.000000001, 1))
    scaler2 = MinMaxScaler(feature_range=(0.000000001, 1))
    scaler3 = MinMaxScaler(feature_range=(0.000000001, 1))
    scaler4 = MinMaxScaler(feature_range=(0.000000001, 1))
    train_sca = pd.DataFrame()
    test_sca = pd.DataFrame()
    train_sca[X_columns] = scaler1.fit_transform(x_train)
    train_sca[Y_columns] = scaler2.fit_transform(y_train)
    test_sca[X_columns] = scaler3.fit_transform(x_test)
    test_sca[Y_columns] = scaler4.fit_transform(y_test)
    return train_sca, test_sca, (scaler1,scaler2, scaler3, scaler4)

x_train, y_train, x_test, y_test, X_columns, Y_columns = load_data()
train_sca, test_sca, (scaler1, scaler2, scaler3, scaler4) = preprocess(x_train, y_train, x_test, y_test, X_columns, Y_columns)
x_train_ts,y_train_ts,x_test_ts,y_test_ts = split_data(train_sca, test_sca, seq, X_columns, Y_columns)
input_size, output_size = len(X_columns), len(Y_columns)

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, model_dim=512, hidden_dim=2048, dropout_rate=0.0):
        super().__init__()
        self.model_dim = model_dim
        self.hidden_dim = hidden_dim
        self.dropout_rate = dropout_rate

        self.linear1 = nn.Linear(self.model_dim, self.hidden_dim)
        self.linear2 = nn.Linear(self.hidden_dim, self.model_dim)
        self.norm = nn.LayerNorm(model_dim)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        output = self.linear2(torch.relu(self.linear1(x)))
        output = self.dropout(output)
        output = self.norm(output + x)
        return output


class GRUAttn(nn.Module):
    def __init__(self, input_dim, model_dim, ff_hidden_dim, num_heads=1, num_blocks=1,
                 dropout_rate=0.1):
        super(GRUAttn, self).__init__()
        self.num_blocks = num_blocks

        gru_hidden_dim = model_dim // 2
        self.gru = nn.GRU(input_dim, gru_hidden_dim, batch_first=True, bidirectional=True)

        for i in range(self.num_blocks):
            self.__setattr__('multihead_attn_{}'.format(i), nn.MultiheadAttention(model_dim, num_heads))
            self.__setattr__('feedforward_{}'.format(i), FeedForward(model_dim=model_dim,
                                                                     hidden_dim=ff_hidden_dim,
                                                                     dropout_rate=dropout_rate))

        self.fc = nn.Linear(model_dim, 1)

    def forward(self, x):
        x, _ = self.gru(x)  # x (B, T, 2 * D/2)
        for i in range(self.num_blocks):
            x, _ = self.__getattr__('multihead_attn_{}'.format(i))(x, x, x)
            x = self.__getattr__('feedforward_{}'.format(i))(x)

        x = self.fc(x)
        return x[:, -1, :]

In [ ]:
def get_model_output(model, input, bs=128):
    with torch.no_grad():
        for i in range(math.ceil(input.shape[0]/bs)):
            Y_pred = model(input[i*bs:(i+1)*bs]).detach().cpu().numpy().reshape(-1, 1)
            if i == 0:
                res = Y_pred
            else:
                res = np.concatenate([res, Y_pred], 0)
    return res

In [ ]:
def train(model, model_path, device, train_iter, x_test, y_test, scaler4):
    myloss = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    import time
    start_time = time.time()

    lossmin = 1
    for epoch in range(epochs):
        for inx, (X, Y) in enumerate(train_iter):
            Y_pred = model(X)
            loss = myloss(Y, Y_pred)
            # 梯度清0
            optimizer.zero_grad()
            # 计算梯度
            loss.backward()
            # 修改权值
            optimizer.step()
            torch.cuda.empty_cache()

        y_test_pred = get_model_output(model, x_test.to(device))
        y_test_pred = scaler4.inverse_transform(y_test_pred)
        loss = mean_absolute_percentage_error(y_test, y_test_pred)
        print(f'*****************epoch{epoch} MAPE {loss}')
        if loss < 1 and loss < lossmin:
            lossmin = loss
            torch.save(model.state_dict(), model_path)

    training_time = time.time() - start_time
    print(f'training_time{training_time}')
    return

In [ ]:
torch_dataset = data.TensorDataset(x_train_ts.to(device), y_train_ts.to(device))
train_iter = data.DataLoader(
    dataset=torch_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0
)


model_path = './mpkl/m15_at_.pkl'
model = GRUAttn(input_size, hidden_size, 32).to(device)


train(model, model_path, device, train_iter, x_test_ts, y_test, scaler4)
model.load_state_dict(torch.load(model_path))



In [ ]:
y_train_pred = get_model_output(model, x_train_ts.to(device))
y_train_pred = scaler2.inverse_transform(y_train_pred)
y_train_pred = pd.DataFrame(columns=['AQ'], data=y_train_pred)

y_test_pred = get_model_output(model, x_test_ts.to(device))
y_test_pred = scaler4.inverse_transform(y_test_pred)
y_test_pred = pd.DataFrame(columns=['AQ'], data=y_test_pred)
print(f'***************************************************')
loss = mean_absolute_percentage_error(y_test, y_test_pred)
print('MAPE:', loss)

In [ ]:
import seaborn as sns
sns.set(context='paper', style='ticks')

fig, axes = plt.subplots(2, 1, figsize=(20, 10))
ax1 = axes[0]
ax2 = axes[1]
fig.subplots_adjust(left=0.1, right=0.8, bottom=0.1, top=0.9, wspace=0.2, hspace=0.3)

# x=train_index[::32]
# y1=y_train['AQ'][::32]
# y2=y_train_pred['AQ'][::32]
lns1 = ax1.plot(y_train[:], label='y_train', marker='v', markevery=1, color='lightcoral')
lns2 = ax1.plot(y_train_pred[:], label='y_train_pred', marker='D', markevery=1, color='dodgerblue')
ax1.tick_params(labelsize=12)
# ax1.set_xlim(0,14208,2000)
# ax1.set_ylim(0,35000, 5000)
# ax1.set_xlabel('Timeindex(8h)',size=20)
# ax1.set_ylabel('AQ(m³/h)',size=20)
ax1.set_title('m1_ridge', size=20, fontweight='bold', loc='left')
lns = lns1 + lns2
labs = [l.get_label() for l in lns]
ax1.legend(lns, labs, loc=1, fontsize=12, ncol=2, frameon=False)

lns1 = ax2.plot(y_test[:], label='y_test', marker='v', markevery=1, color='lightcoral')
lns2 = ax2.plot(y_test_pred[:], label='y_test_pred', marker='D', markevery=1, color='dodgerblue')
ax2.tick_params(labelsize=12)
# ax1.set_xlim(0,14208,2000)
# ax1.set_ylim(0,35000, 5000)
# ax1.set_xlabel('Timeindex(8h)',size=20)
# ax1.set_ylabel('AQ(m³/h)',size=20)
ax2.set_title('m1_ridge', size=20, fontweight='bold', loc='left')
lns = lns1 + lns2
labs = [l.get_label() for l in lns]
ax2.legend(lns, labs, loc=1, fontsize=12, ncol=2, frameon=False)

frame = pd.DataFrame()
frame['ytrain'] = y_train_pred
frame['ytest'] = y_test_pred
frame.to_excel('./modelresult/m15_at.xlsx')
plt.show()